In [1]:
require(ggplot2)
require(ggpubr)
require(dplyr)

Loading required package: ggplot2

Loading required package: ggpubr

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
species <- c('Hsap', 'Mmus')

In [18]:
for (s in species){
    f1 <- read.delim(paste0(s,'.ohno.bar.txt'), row.names = 1, header = F)[,1:2]
    f2 <- read.delim(paste0(s,'.SSDpara.bar.txt'), row.names = 1, header = F)[,1:2]
    f1$FDR <- 'ns'
    f2$FDR <- 'ns'
    f3 <- read.delim(paste0(s,'.ohno.protein_class.overrep.txt'), check.names = F)[,c(1,5,8)]
    f3 <- f3[which(f3[,2] == '+'),] # only positive association will be plotted
    f4 <- read.delim(paste0(s,'.SSDpara.protein_class.overrep.txt'), check.names = F)[,c(1,5,8)]
    f4 <- f4[which(f4[,2] == '+'),]
    f1$FDR <- vapply(f1$V2, FUN = function(x){
        if (x %in% f3[,1]){
            return(f3[f3$`PANTHER Protein Class` == x, 3])
        } else {
            return(1)
        }
    }, FUN.VALUE = double(1))
    
    f2$FDR <- vapply(f2$V2, FUN = function(x){
        if (x %in% f4[,1]){
            return(f4[f4$`PANTHER Protein Class` == x, 3])
        } else {
            return(1)
        }
    
    }, FUN.VALUE = double(1))
    
    
    f1$type <- 'WGD'
    f2$type <- 'SSD'
    res <- rbind(f1,f2)
    res$V2 <- factor(res$V2, levels = res[order(res[res$type == 'WGD', "V3"], decreasing = T), "V2"])
    res$type <- factor(res$type, levels= c('WGD', 'SSD'))
    res <- res %>% filter(V2 != 'No PANTHER category is assigned (UNCLASSIFIED)')
    
    res <- res %>% mutate(Significance = case_when(
        FDR < 0.001 ~ "****",
        FDR < 0.001 ~ "***",  # Highly significant
        FDR < 0.01  ~ "**",   # Very significant
        FDR < 0.05  ~ "*",    # Significant
        TRUE       ~ "ns"      # Not significant (empty string)
      ))

    res <- res %>% mutate(Broad_Class = case_when(
        grepl("extracellular matrix protein|cytoskeletal protein|structural protein|cell junction protein", V2) ~ "Structural Proteins",
        grepl("metabolite interconversion enzyme|protein modifying enzyme", V2) ~ "Enzymes",
        grepl("gene-specific transcriptional regulator|chromatin/chromatin-binding|scaffold/adaptor protein|protein-binding activity modulator", V2) ~ "Regulatory Proteins",
        grepl("transporter|transfer/carrier protein|membrane traffic protein", V2) ~ "Transport & Trafficking Proteins",
        grepl("transmembrane signal receptor|intercellular signal molecule|calcium-binding protein", V2) ~ "Signal Transduction",
        grepl("cell adhesion molecule|chaperone", V2) ~ "Adhesion & Interaction Proteins",
        grepl("DNA metabolism protein|RNA metabolism protein|translational protein", V2) ~ "Genetic Information Processing",
        grepl("defense/immunity protein", V2) ~ "Defense & Immunity",
        TRUE ~ "Unclassified"
      ),
      Color = case_when(
        Broad_Class == "Structural Proteins" ~ "#1f77b4",  # Blue
        Broad_Class == "Enzymes" ~ "#ff7f0e",  # Orange
        Broad_Class == "Regulatory Proteins" ~ "#2ca02c",  # Green
        Broad_Class == "Transport & Trafficking Proteins" ~ "#d62728",  # Red
        Broad_Class == "Signal Transduction" ~ "#9467bd",  # Purple
        Broad_Class == "Adhesion & Interaction Proteins" ~ "#8c564b",  # Brown
        Broad_Class == "Genetic Information Processing" ~ "#e377c2",  # Pink
        Broad_Class == "Defense & Immunity" ~ "#7f7f7f",  # Gray
        Broad_Class == "Unclassified" ~ "#bcbd22"  # Yellow-Green
      ))
    p <- ggbarplot(res, x = "V2", y = "V3", label = res$Significance, label.pos = "out", fill = "Broad_Class") + 
        facet_wrap(~type, ncol = 1) + 
        theme(axis.text.x = element_text(angle = 90, hjust = 1)) 
        #theme(legend.position = "none")
    ggsave(paste0(s, '.protein_class.pdf'), p, width = 7.5, height = 10)
}